## Document Reviewer with Palm

### Goals
We want to explore how Cohere perform as a document reviewer. Given a technical documentation, can Cohere generate an improved version of it better than other LLMs?

### Techniques


- all-in-one prompt: we first start by putting all the desired outcome in a single prompt
- multiple prompts by roles: we feed the document into individual prompts that does only one thing. The completion is then fed into another prompt until it goes through all the chain
- decomposition: we ask the LLM to decompose the document first, and then convert the documents content to questions instead. Then we ask the LLM to answer the questions and then reconstruct the document.

### Loading

We first load our sample document to improve.

In [1]:
document = """
# Use Constant


## Avoid const*.go

Avoid naming files with any variation of `const.go` to store constants.

Each constant serves a purpose, and they probably should belong to domain layer.


## Don't uppercase constant


```go
// Bad
const CODE_NOT_FOUND = 404


// Good
const CodeNotFound = 404
```

## Prefer constructor over constant


There are many times where constructor proves to be more useful than plain constant.

For example, you may have a constant to declare the expiry time of a campaign:


```go
var CampaignExpiredAt = time.Date(2023, time.January, 1, 0, 0, 0, 0, time.Local)
```

However, it doesn't serve much purpose there. Instead, add an accompanying function that utilizes it.

```go
package main

import (
	"fmt"
	"time"
)

var CampaignExpiredAt = time.Date(2023, time.January, 1, 0, 0, 0, 0, time.Local)

func CampaignActive(now time.Time) bool {
	return now.Before(CampaignExpiredAt)
}

func main() {
	now := time.Now()
	fmt.Println(now)
	fmt.Println(CampaignActive(now))
}
```


Other examples includes
- base url
- expiry duration
- cache prefix key
"""

In [2]:
from IPython.display import Markdown

In [3]:
Markdown(document)


# Use Constant


## Avoid const*.go

Avoid naming files with any variation of `const.go` to store constants.

Each constant serves a purpose, and they probably should belong to domain layer.


## Don't uppercase constant


```go
// Bad
const CODE_NOT_FOUND = 404


// Good
const CodeNotFound = 404
```

## Prefer constructor over constant


There are many times where constructor proves to be more useful than plain constant.

For example, you may have a constant to declare the expiry time of a campaign:


```go
var CampaignExpiredAt = time.Date(2023, time.January, 1, 0, 0, 0, 0, time.Local)
```

However, it doesn't serve much purpose there. Instead, add an accompanying function that utilizes it.

```go
package main

import (
	"fmt"
	"time"
)

var CampaignExpiredAt = time.Date(2023, time.January, 1, 0, 0, 0, 0, time.Local)

func CampaignActive(now time.Time) bool {
	return now.Before(CampaignExpiredAt)
}

func main() {
	now := time.Now()
	fmt.Println(now)
	fmt.Println(CampaignActive(now))
}
```


Other examples includes
- base url
- expiry duration
- cache prefix key


In [4]:
import app.palm as palm

/Users/alextanhongpin/Documents/python/python-prompting/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
prompt = """
You are an experienced technical writer.
Given a technical documentation, improve the content by elaborating on the key insights.

Document:
{document}
"""

In [6]:
completion = palm.generate_text(prompt.format(document=document))
completion

"**Avoid const*.go**\n\nNaming files with any variation of `const.go` to store constants is not recommended. Each constant serves a purpose, and they probably should belong to the domain layer. For example, a constant that represents the expiry time of a campaign should be defined in the campaign package, not in a separate file called `const.go`.\n\n**Don't uppercase constant**\n\nGolang's style guide recommends that constants be written in all lowercase letters with underscores between words. For example, `const CodeNotFound = 404`. This makes constants easier to read and understand.\n\n**Prefer constructor over constant**\n\nIn some cases, a constructor can be more useful than a plain constant. For example, you may have a constant to declare the expiry time of a campaign:\n\n```go\nvar CampaignExpiredAt = time.Date(2023, time.January, 1, 0, 0, 0, 0, time.Local)\n```\n\nHowever, this constant doesn't serve much purpose on its own. It would be more useful to have a function that utiliz

In [7]:
Markdown(completion)

**Avoid const*.go**

Naming files with any variation of `const.go` to store constants is not recommended. Each constant serves a purpose, and they probably should belong to the domain layer. For example, a constant that represents the expiry time of a campaign should be defined in the campaign package, not in a separate file called `const.go`.

**Don't uppercase constant**

Golang's style guide recommends that constants be written in all lowercase letters with underscores between words. For example, `const CodeNotFound = 404`. This makes constants easier to read and understand.

**Prefer constructor over constant**

In some cases, a constructor can be more useful than a plain constant. For example, you may have a constant to declare the expiry time of a campaign:

```go
var CampaignExpiredAt = time.Date(2023, time.January, 1, 0, 0, 0, 0, time.Local)
```

However, this constant doesn't serve much purpose on its own. It would be more useful to have a function that utilizes the constant, such as:

```go
func CampaignActive(now time.Time) bool {
	return now.Before(CampaignExpiredAt)
}
```

This function takes a `time.Time` object as its input and returns a boolean value indicating whether the campaign is still active. This is a more useful representation of the constant than the constant itself.

**Other examples of constants**

In addition to the examples mentioned above, there are many other cases where constants can be useful. Some common examples include:

* Base URL: A constant that represents the base URL of a web service can be used to construct requests to that service.
* Expiry duration: A constant that represents the duration for which a cache entry should be valid can be used to ensure that cached data is not stale.
* Cache prefix key: A constant that represents the prefix for cache keys can be used to ensure that cache keys are unique.

By following these guidelines, you can use constants effectively in your Golang code.

In [8]:
prompt = """
You are an experienced technical writer.
Given a technical documentation, improve the content by elaborating on the key insights.
Keep the code samples unchanged. Return only the improved document.

Document:
{document}
"""

completion = palm.generate_text(prompt.format(document=document))
completion

'# Use Constant\n\nConstants are useful for storing values that will not change throughout the lifetime of your program. They can be declared at the package level or within a function.\n\n## Avoid const*.go\n\nAvoid naming files with any variation of `const.go` to store constants.\n\nEach constant serves a purpose, and they probably should belong to domain layer. For example, if you have a constant for the expiry time of a campaign, it would make sense to store that constant in the campaign package.\n\n## Don\'t uppercase constant\n\nGolang has a convention of using lowercased names for constants. This makes it easier to read and understand code.\n\n```go\n// Bad\nconst CODE_NOT_FOUND = 404\n\n\n// Good\nconst CodeNotFound = 404\n```\n\n## Prefer constructor over constant\n\nThere are many times where constructor proves to be more useful than plain constant.\n\nFor example, you may have a constant to declare the expiry time of a campaign:\n\n```go\nvar CampaignExpiredAt = time.Date(202

In [9]:
Markdown(completion)

# Use Constant

Constants are useful for storing values that will not change throughout the lifetime of your program. They can be declared at the package level or within a function.

## Avoid const*.go

Avoid naming files with any variation of `const.go` to store constants.

Each constant serves a purpose, and they probably should belong to domain layer. For example, if you have a constant for the expiry time of a campaign, it would make sense to store that constant in the campaign package.

## Don't uppercase constant

Golang has a convention of using lowercased names for constants. This makes it easier to read and understand code.

```go
// Bad
const CODE_NOT_FOUND = 404


// Good
const CodeNotFound = 404
```

## Prefer constructor over constant

There are many times where constructor proves to be more useful than plain constant.

For example, you may have a constant to declare the expiry time of a campaign:

```go
var CampaignExpiredAt = time.Date(2023, time.January, 1, 0, 0, 0, 0, time.Local)
```

However, it doesn't serve much purpose there. Instead, add an accompanying function that utilizes it.

```go
package main

import (
	"fmt"
	"time"
)

var CampaignExpiredAt = time.Date(2023, time.January, 1, 0, 0, 0, 0, time.Local)

func CampaignActive(now time.Time) bool {
	return now.Before(CampaignExpiredAt)
}

func main() {
	now := time.Now()
	fmt.Println(now)
	fmt.Println(CampaignActive(now))
}
```

This constructor takes a `time.Time` argument and returns a boolean value indicating whether the campaign is still active. This is more useful than the constant because it allows us to check the campaign's expiry time at runtime.

Other examples of constants that can be replaced with constructors include:

- base url
- expiry duration
- cache prefix key

By using constructors instead of constants, we can make our code more readable and maintainable.

In [22]:
prompt = """
You are provided a technical document. Think step by step.
1. extract the key topic of this document. 
2. suggest points about this topic that is not already present in the document
3. ask critical and relevant questions about the topics
4. attempt to answer the questions about the topic, providing clear and simple explanation.

Summarize all of it into an improved version of the document. Return only the improved version in Markdown.

Document:
----------------------------------------
{document}
----------------------------------------
"""

completion = palm.generate_text(prompt.format(document=document))
completion

'## Use Constant\n\n### Avoid const*.go\n\nAvoid naming files with any variation of `const.go` to store constants.\n\nEach constant serves a purpose, and they probably should belong to domain layer.\n\n### Don\'t uppercase constant\n\n```go\n// Bad\nconst CODE_NOT_FOUND = 404\n\n\n// Good\nconst CodeNotFound = 404\n```\n\n### Prefer constructor over constant\n\nThere are many times where constructor proves to be more useful than plain constant.\n\nFor example, you may have a constant to declare the expiry time of a campaign:\n\n```go\nvar CampaignExpiredAt = time.Date(2023, time.January, 1, 0, 0, 0, 0, time.Local)\n```\n\nHowever, it doesn\'t serve much purpose there. Instead, add an accompanying function that utilizes it.\n\n```go\npackage main\n\nimport (\n\t"fmt"\n\t"time"\n)\n\nvar CampaignExpiredAt = time.Date(2023, time.January, 1, 0, 0, 0, 0, time.Local)\n\nfunc CampaignActive(now time.Time) bool {\n\treturn now.Before(CampaignExpiredAt)\n}\n\nfunc main() {\n\tnow := time.Now()\

In [23]:
Markdown(completion)

## Use Constant

### Avoid const*.go

Avoid naming files with any variation of `const.go` to store constants.

Each constant serves a purpose, and they probably should belong to domain layer.

### Don't uppercase constant

```go
// Bad
const CODE_NOT_FOUND = 404


// Good
const CodeNotFound = 404
```

### Prefer constructor over constant

There are many times where constructor proves to be more useful than plain constant.

For example, you may have a constant to declare the expiry time of a campaign:

```go
var CampaignExpiredAt = time.Date(2023, time.January, 1, 0, 0, 0, 0, time.Local)
```

However, it doesn't serve much purpose there. Instead, add an accompanying function that utilizes it.

```go
package main

import (
	"fmt"
	"time"
)

var CampaignExpiredAt = time.Date(2023, time.January, 1, 0, 0, 0, 0, time.Local)

func CampaignActive(now time.Time) bool {
	return now.Before(CampaignExpiredAt)
}

func main() {
	now := time.Now()
	fmt.Println(now)
	fmt.Println(CampaignActive(now))
}
```

Other examples includes
- base url
- expiry duration
- cache prefix key

### Additional points

* Use `iota` to generate sequential integer constants.
* Use `string` constants for non-numeric values.
* Use `json` tags to document constants.
* Use `go generate` to generate constants from a .csv file.

### Critical questions

* What is the difference between a constant and a variable?
* Why should I use constructors over constants?
* What are some best practices for using constants?

### Clear and simple answers

* A constant is a value that cannot be changed during program execution, while a variable can be changed.
* Constructors are more useful than constants because they can be used to create new values of the constant type, while constants cannot be changed.
* Some best practices for using constants include: using `iota` to generate sequential integer constants, using `string` constants for non-numeric values, using `json` tags to document constants, and using `go generate` to generate constants from a .csv file.

In [24]:
prompt = """
You are given summaries of a technical document on a specific topic.
Write an ADR about this topic based on the provided summary. 
Return the ADR in Markdown format.

Summaries:
{summary}
"""
completion = palm.generate_text(prompt.format(summary=completion))
completion

'## ADR: Use Constant\n\n### Objective\n\nThe objective of this ADR is to define the best practices for using constants in Golang.\n\n### Background\n\nConstants are values that cannot be changed during program execution. They are typically used to represent values that are known at compile time, such as pi or the number of days in a year.\n\n### Proposed Solution\n\nThe following are some best practices for using constants in Golang:\n\n* Use `iota` to generate sequential integer constants.\n* Use `string` constants for non-numeric values.\n* Use `json` tags to document constants.\n* Use `go generate` to generate constants from a .csv file.\n\n### Rationale\n\nUsing `iota` to generate sequential integer constants can help to improve the readability and maintainability of your code. Using `string` constants for non-numeric values can help to avoid errors caused by typos. Using `json` tags to document constants can help to make your code more readable and understandable. Using `go gener

In [25]:
Markdown(completion)

## ADR: Use Constant

### Objective

The objective of this ADR is to define the best practices for using constants in Golang.

### Background

Constants are values that cannot be changed during program execution. They are typically used to represent values that are known at compile time, such as pi or the number of days in a year.

### Proposed Solution

The following are some best practices for using constants in Golang:

* Use `iota` to generate sequential integer constants.
* Use `string` constants for non-numeric values.
* Use `json` tags to document constants.
* Use `go generate` to generate constants from a .csv file.

### Rationale

Using `iota` to generate sequential integer constants can help to improve the readability and maintainability of your code. Using `string` constants for non-numeric values can help to avoid errors caused by typos. Using `json` tags to document constants can help to make your code more readable and understandable. Using `go generate` to generate constants from a .csv file can help to save time and effort.

### Implementation Plan

The following are the steps involved in implementing the proposed solution:

1. Add the following import statements to your Golang project:

```
import "fmt"
import "time"
```

2. Define a constant to represent the number of days in a year:

```
const DaysInYear = 365
```

3. Use `iota` to generate a sequence of integer constants:

```
const (
    Day1 = iota
    Day2
    Day3
    Day4
    Day5
)
```

4. Define a constant to represent the base URL of your API:

```
const BaseURL = "https://api.example.com"
```

5. Use `json` tags to document your constants:

```
const (
    // ExpiryDuration is the duration after which a cache entry expires.
    ExpiryDuration = time.Hour * 24
)
```

6. Use `go generate` to generate constants from a .csv file:

```
//go:generate go run github.com/golang/tools/cmd/stringer -type=Currency
```

### Testing Plan

The following tests can be used to test the proposed solution:

* Test that the constants are defined correctly.
* Test that the constants are used correctly in your code.
* Test that the constants are documented correctly.

### Deployment Plan

The following steps are involved in deploying the proposed solution:

1. Commit the changes to your code repository.
2. Push the changes to your remote repository.
3. Deploy the changes to your production environment.

### Monitoring Plan

The following metrics can be used to monitor the proposed solution:

* The number of constants defined in your code.
* The number of times the constants are used in your code.
* The number of errors caused by typos in your constants.

### Conclusion

The proposed solution provides a number of benefits, including:

* Improved readability and maintainability of your code.
* Reduced risk of errors caused by typos.
* Improved documentation of your code.
* Reduced time and effort required to generate constants.

I recommend that you adopt the proposed solution in your Golang projects.

In [18]:
completion = palm.generate_text("""summarize the format of an RFC document""")
completion

"* **Title:** The title of the RFC document should be short and concise, and should accurately reflect the contents of the document.\n* **Status:** The status of the RFC document should be indicated at the beginning of the document. The possible statuses are:\n    * **Proposed Standard:** This is the highest status that an RFC document can achieve. A Proposed Standard RFC document is considered to be a stable and well-tested specification that is ready for widespread deployment.\n    * **Draft Standard:** A Draft Standard RFC document is a proposed standard that is still under development. Draft Standard RFC documents are not considered to be stable or well-tested, and should not be deployed in production environments.\n    * **Informational:** An Informational RFC document is a document that provides information about a particular topic, but does not specify a standard. Informational RFC documents are not intended to be implemented, and should not be deployed in production environment

In [19]:
Markdown(completion)

* **Title:** The title of the RFC document should be short and concise, and should accurately reflect the contents of the document.
* **Status:** The status of the RFC document should be indicated at the beginning of the document. The possible statuses are:
    * **Proposed Standard:** This is the highest status that an RFC document can achieve. A Proposed Standard RFC document is considered to be a stable and well-tested specification that is ready for widespread deployment.
    * **Draft Standard:** A Draft Standard RFC document is a proposed standard that is still under development. Draft Standard RFC documents are not considered to be stable or well-tested, and should not be deployed in production environments.
    * **Informational:** An Informational RFC document is a document that provides information about a particular topic, but does not specify a standard. Informational RFC documents are not intended to be implemented, and should not be deployed in production environments.
* **Abstract:** The abstract of the RFC document should provide a brief overview of the contents of the document.
* **Body:** The body of the RFC document should contain the detailed specifications of the proposed standard. The body should be written in a clear and concise manner, and should be easy to understand.
* **References:** The references section of the RFC document should list any other RFC documents that are relevant to the proposed standard.
* **Security Considerations:** The security considerations section of the RFC document should discuss any potential security risks associated with the proposed standard.
* **Authors' Addresses:** The authors' addresses section of the RFC document should list the contact information for the authors of the document.

For more information on the format of RFC documents, please refer to the [RFC 2026](https://tools.ietf.org/html/rfc2026) specification.

In [20]:
completion = palm.generate_text("""summarize the format of an ADR document""")
completion

'**ADR Document Format**\n\n* **Title:** The title of the ADR document should be clear and concise, and should accurately reflect the content of the document.\n* **Introduction:** The introduction should provide a brief overview of the ADR process and the purpose of the document.\n* **Body:** The body of the ADR document should include the following information:\n    * The parties to the ADR process\n    * The issues to be resolved through ADR\n    * The procedures to be used for ADR\n    * The timeline for ADR\n    * The costs of ADR\n* **Conclusion:** The conclusion of the ADR document should include a statement of agreement by the parties to the process, and a plan for implementation of the ADR agreement.\n\n**Sample ADR Document**\n\n* **Title:** Agreement to Mediate\n\n* **Introduction:**\n\nThis Agreement to Mediate (the "Agreement") is entered into on [DATE] by and between [PARTY A] (the "Party A") and [PARTY B] (the "Party B").\n\n* **Body:**\n\n1. The Parties agree to submit t

In [21]:
Markdown(completion)

**ADR Document Format**

* **Title:** The title of the ADR document should be clear and concise, and should accurately reflect the content of the document.
* **Introduction:** The introduction should provide a brief overview of the ADR process and the purpose of the document.
* **Body:** The body of the ADR document should include the following information:
    * The parties to the ADR process
    * The issues to be resolved through ADR
    * The procedures to be used for ADR
    * The timeline for ADR
    * The costs of ADR
* **Conclusion:** The conclusion of the ADR document should include a statement of agreement by the parties to the process, and a plan for implementation of the ADR agreement.

**Sample ADR Document**

* **Title:** Agreement to Mediate

* **Introduction:**

This Agreement to Mediate (the "Agreement") is entered into on [DATE] by and between [PARTY A] (the "Party A") and [PARTY B] (the "Party B").

* **Body:**

1. The Parties agree to submit the following dispute to mediation:
    * [BRIEF DESCRIPTION OF DISPUTE]

2. The Parties agree to use the following mediation procedures:
    * [LIST OF MEDIATION PROCEDURES]

3. The Parties agree to mediate the dispute within [TIME PERIOD] days of the date of this Agreement.

4. The Parties agree to share the costs of mediation equally.

* **Conclusion:**

5. The Parties agree that any settlement reached through mediation will be in writing and signed by both Parties.

6. This Agreement may be modified only by a written agreement signed by both Parties.

7. This Agreement will be governed by and construed in accordance with the laws of the State of [STATE].

8. This Agreement constitutes the entire agreement between the Parties with respect to the subject matter hereof and supersedes all prior or contemporaneous communications, representations, or agreements, whether oral or written.

9. This Agreement may be executed in one or more counterparts, each of which will be deemed an original, but all of which together will constitute one and the same instrument.

[SIGNATURES OF PARTIES]